In [ ]:
import pandas as pd
import numpy  as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

import re
import string

In [ ]:
df_fake_nws = pd.read_csv("/content/Fake.csv")
df_true_nws = pd.read_csv("/content/True.csv")

In [ ]:
df_fake_nws.head(10)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017"
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017"
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017"
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017"
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017"


In [ ]:
df_true_nws.head(10)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017"
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",politicsNews,"December 29, 2017"
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
9,Alabama official to certify Senator-elect Jone...,WASHINGTON (Reuters) - Alabama Secretary of St...,politicsNews,"December 28, 2017"


In [ ]:
df_fake_nws["class"] = 0
df_true_nws["class"] = 1

In [ ]:
df_fake_nws.shape, df_true_nws.shape

((23481, 5), (21417, 5))

In [ ]:
df_fake_nws_manual_test = df_fake_nws.tail(10)
for i in range(23480,23470,-1):
  df_fake_nws.drop([i],axis = 0, inplace = True)

In [ ]:
df_true_nws_manual_test = df_true_nws.tail(10)
for i in range(21416,23470,-1):
  df_true_nws.drop([i],axis = 0, inplace = True)

In [ ]:
df_merge = pd.concat([df_fake_nws,df_true_nws],axis=0)
df_merge["text"].sample()

,text
14235,"No matter which candidate you support, this mo..."


In [ ]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [ ]:
df = df_merge.drop(["title","subject","date"],axis= 1)

In [ ]:
df.isnull().sum()

,0
text,0
class,0


In [ ]:
df= df.sample(frac= 1)


In [ ]:
df.reset_index(inplace= True)
df.drop(["index"],axis=1,inplace = True)

In [ ]:
df.columns

Index(['text', 'class'], dtype='object')

In [ ]:
df.sample()

,text,class
43933,(Reuters) - Ailing Puerto Rico would receive $...,1


In [ ]:
import re
import string

def wordopt(text):
    text = text.lower()  # Convert text to lowercase

    # 1. Remove text inside square brackets (e.g., "[example]")
    text = re.sub(r'\[.*?\]', '', text)

    # 2. Remove all non-word characters except spaces (e.g., special symbols like "#, %, $, etc.")
    text = re.sub(r'\W', ' ', text)

    # 3. Remove single characters (e.g., 'a', 'b', 'c' that are isolated)
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

    # 4. Remove URLs (e.g., "https://example.com" or "www.example.com")
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # 5. Remove any text within angle brackets (e.g., "<example>")
    text = re.sub(r'<.*?>+', '', text)

    # 6. Remove punctuation using `string.punctuation`
    text = re.sub(r'[{}]'.format(re.escape(string.punctuation)), '', text)

    # 7. Remove newlines (`\n`)
    text = re.sub(r'\n', '', text)

    # 8. Remove words containing numbers (e.g., "word123")
    text = re.sub(r'\w*\d\w*', '', text)

    return text


In [ ]:
df["text"] = df["text"].apply(wordopt)

In [ ]:
x = df["text"]
y = df["class"]

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.25)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vector = TfidfVectorizer()
xv_train = vector.fit_transform(x_train)
xv_test = vector.transform(x_test)

In [ ]:
print(xv_train[:5].toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(xv_train,y_train)

LogisticRegression()

In [ ]:
lr_pred = lr.predict(xv_test)

In [ ]:
lr.score(xv_test , y_test)

0.9870789520584566

In [ ]:
print(classification_report(y_test,lr_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5772
           1       0.99      0.99      0.99      5450

    accuracy                           0.99     11222
   macro avg       0.99      0.99      0.99     11222
weighted avg       0.99      0.99      0.99     11222



In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(xv_train,y_train)

DecisionTreeClassifier(random_state=42)

In [ ]:
dt_pred = dt.predict(xv_test)
dt.score(xv_test ,y_test)

0.9960791302798075

In [ ]:
print(classification_report(y_test, dt_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5772
           1       1.00      1.00      1.00      5450

    accuracy                           1.00     11222
   macro avg       1.00      1.00      1.00     11222
weighted avg       1.00      1.00      1.00     11222



In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(xv_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
rf_pred = rf_model.predict(xv_test)
rf_model.score(xv_test,y_test)

0.9892176082694707

In [ ]:
print(classification_report(y_test, rf_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5772
           1       0.99      0.99      0.99      5450

    accuracy                           0.99     11222
   macro avg       0.99      0.99      0.99     11222
weighted avg       0.99      0.99      0.99     11222



In [ ]:
def output_lable(n):
    if n == 0:
        return "Fake News❌"
    elif n == 1:
        return "True News✅"

def manual_testing(news):
    testing_news = {"text": [news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vector.transform(new_x_test)

    # Predictions from different models
    lr_pred = lr.predict(new_xv_test)  # Logistic Regression
    dt_pred = dt.predict(new_xv_test)  # Decision Tree
    rfc_pred = rf_model.predict(new_xv_test)  # Random Forest

    # Print predictions
    print("""
    LR Prediction: {}
    DT Prediction: {}
    RFC Prediction: {}
    """.format(
        output_lable(lr_pred[0]),
        output_lable(dt_pred[0]),
        output_lable(rfc_pred[0])
    ))

In [ ]:
news = str(input())
manual_testing()